In [43]:
!pip install geemap        # --> Google Earth Engine (GEE)
!pip install geopandas      #--> Geospatial data 
%pip install scikit-learn   #--> Machine learning 

# Talking with local terminal on your computer
import os

# GEE
import ee
from geemap.conversion import js_snippet_to_py
import geemap
from geemap import ml

# Geospatial data manipulation and analysis tools
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import tensorflow as tf
import seaborn as sns
import time

# Machine learning with sci-kit learn
from sklearn import ensemble
from sklearn.model_selection import train_test_split, learning_curve, validation_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.datasets import (make_blobs,
                              make_circles,
                              make_moons)
from sklearn.cluster import KMeans, SpectralClustering, DBSCAN

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Check which prorcessor unit we are using
tf.test.gpu_device_name()

In [ ]:
# Initialize access to GEE
Map = geemap.Map()

# 0) Follow external link 
# 1) Choose project name
# 2) "Generate token"
# 3) Choose account
# 4) Give access (check of both boxes)
# 5) Copy authorization code
# 6) Add authorization code below

In [ ]:
# Metadata on hardware, software etc. 
geemap.Report()

In [ ]:
# Make a cloud-free Sentinel-2 composite (from raw imagery).
# Source code: https://developers.google.com/earth-engine/guides/ic_info. Accessed: 16.02.2023

point = ee.Geometry.Point([-69.740859, 43.544235]) # Center around a specified point

image = (
    ee.ImageCollection('COPERNICUS/S2_SR')         # Dataset from GEE
    .filterBounds(point)                           # Filter satellite-images around our region of interest (ROI)
    .filterDate('2019-06-02','2019-10-01')         # Filter time
    .sort('CLOUDY_PIXEL_PERCENTAGE')               # fiter for cloud percentage
    .first()                                       # Take the image with least cloud coverage
    .select('B[2-4]','B[8]')                       # Select bands B2, B3, B4, and B8 as features
)

print('Date image was taken in the format YYYY-MM-DD:', ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()) # Show the image-date
print('Cloud percentage: ', image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo())

# Image pre-pocessing

##Sun-glint correction


- Adaptation from https://www.youtube.com/watch?v=_V0X1LdSta4. Accessed 10.02.2023.
- The python dicitonary needs ** in front of the dictionary in order pass keyword arguments into it. This does not go automatically with the 'js_snippet_to_py'. See https://realpython.com/python-kwargs-and-args/ for complete explenation. Accessed 16.02.2023. 

In [ ]:
Map.set_center(-69.740859, 43.544235, zoom=9)

# Show sentinel-2 tile in map
vis_params = {"bands": [ 'B2', 'B3', 'B4'], "min": 0, "max": 1500, "gamma": 1.5}
Map.addLayer(image, vis_params, 'Sentinel-2 images')

Map 
# Use toolbar on the left side, e.g., " Draw a rectangle" over a sight 
# to extract NIR values (min and random) from two pixels. 
# Areas of deep waters are recommended.

In [ ]:
Map.draw_features                                # Collects the drawn rectangle(s)
glint = ee.FeatureCollection(Map.draw_features)  # Stores the rectangle(s) geometry to extract NIR-values from

def deglint(img):
  '''
  Sun-glint correction of image
  '''
  # Blue (B2) and NIR (B8) 
  # linearfitB gets linear relationship (slope) and offset between the Blue and NIR bands
  linearfitB = img.select(['B8','B2']).reduceRegion(
    **{
    'reducer': ee.Reducer.linearFit(),  
    'geometry': glint,
    'scale':10,
    'maxPixels':1e9
    }) 

  # Green (B3) and NIR (B8)
  # linearfitG gets linear relationship (slope) and offset between the Green and NIR bands  
  linearfitG = img.select(['B8','B3']).reduceRegion(
     **{
      'reducer': ee.Reducer.linearFit(),
      'geometry': glint,
      'scale':10,
      'maxPixels':1e9
      })  
  # Red and NIR
  # linearfitR gets linear relationship (slope) and offset between the Red and NIR bands
  linearfitR = img.select(['B8','B4']).reduceRegion(
    **{
      'reducer': ee.Reducer.linearFit(),
      'geometry': glint,
      'scale':10,
      'maxPixels':1e9
      })
  
  
  slopeImage = ee.Dictionary(            # Collects the slopes in a dictionary 
  {
      'Blue': linearfitB.get('scale'),
      'Green': linearfitG.get('scale'),
      'Red': linearfitR.get('scale')
      }).toImage()                        # Converts dicitonary to image
      
  # Define minimum NIR value
  minNIR = img.select('B8').reduceRegion(ee.Reducer.min(), glint).toImage()

  # Deglint formula
  return img.select(['B2','B3', 'B4']).subtract(slopeImage.multiply(img.select('B8').subtract(minNIR)))

# Apply deglint function
S2_deglint = deglint(image) 

# Might get a HttpError here stating: 
# "Too many pixels in the region. Found 11099489, but maxPixels allows only 10000000."
# The problem is that the rectangle created to extract NIR-values is too big. 
# Thus, create a new and smaller rectangle, or for future recommendations,
# create a error-statement (using if-else) to where this is clearly explained or something similar. 

# Show deglinted image in map
vis_params_deglint = {"bands": [ 'B2', 'B3','B4'], "min": 0, "max": 1500, 'gamma':1.5}

Map.addLayer(S2_deglint, vis_params_deglint, 'sun-glint corrected Sent-2')
Map

# Feature collection for pixel-based classification (6 features)


From the raw sentinel-2 tile, B2, B3, B4, and B8 were collected. For the pixel-based analysis we chose to collect the normalized difference vegetation index (NDVI) and bathymetry. In total 6 features. 

### Add NDVI-band to image

In [ ]:
# Calculating NDVI
nir = image.select('B8')
red = image.select('B4')
ndvi_image = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

# Add NIR-band to deglinted image
S2_deglint = S2_deglint.addBands(nir)

# Add NDVI to RGB-image and deglinted image
S2_deglint = S2_deglint.addBands(ndvi_image)
image = image.addBands(ndvi_image)

# sent2019 = ee.Image('COPERNICUS/S2/20190719T153601_20190719T153707_T19TDJ')

### Add bathemerty band to image

In [ ]:
Map # Draw region of interest


In [ ]:
# Store geometry of region of interest in 'roi'
roi = ee.FeatureCollection(Map.draw_last_feature)

#Citations:
# Amante, C. and B. W. Eakins, ETOPO1 1 Arc-Minute Global Relief Model: Procedures, Data Sources and Analysis. 
# NOAA Technical Memorandum NESDIS NGDC-24, 19 pp, March 2009.

# Import elevation dataset from GEE
dataset = ee.Image('NOAA/NGDC/ETOPO1')
bathymetry_clipped = dataset.clip(roi)

# Extract band 'bedrock' and rename it bathymetry for convenience
bathymetry = bathymetry_clipped.select('bedrock').rename('bathymetry')

# Set visualization parameters
bathymetryVis = {
  'min': -7000.0,
  'max': 3000.0,
  'palette': ['011de2', 'afafaf', '3603ff', 'fff477', 'b42109'],
}

# Add bathymetry band to deglinted and raw image
S2_deglint = S2_deglint.addBands(bathymetry)
image = image.addBands(bathymetry)

S2_deglint.bandNames()
# Add NDVI-image to map
#ndviParams = {min: -1, max: 1, 'palette': ['blue', 'white', 'green']}
#Map.addLayer(ndvi_image, ndviParams, 'NDVI image')
# Add bathymetry data on the map
#Map.addLayer(bathymetry, bathymetryVis, 'bathymetry')

# Feature collection for Object-based image analysis classification (11 features).

For the OBIA 5 additional feautres are added to the dataset used to classify seagrass from seaweed. These are obtained by creating super-pixels and then extracting their characteristics. That is: area, clusters, height, perimeter, and width. 

1) Create super-pixels with SNIC

In [ ]:
# Select seed-pixels for clustering, spaced at least 36 pixels apart. Clipped to region of interest.  
seeds = ee.Algorithms.Image.Segmentation.seedGrid(36).clip(roi)

# Run SNIC
snic = ee.Algorithms.Image.Segmentation.SNIC(**{
  'image': S2_deglint,                               
  'size': 32,                                      # 
  'compactness': 5,
  'connectivity': 8,
  'neighborhoodSize':256,
  'seeds': seeds
}).select(['B2_mean', 'B3_mean', 'B4_mean', 'B8_mean','clusters', 'bathymetry_mean','NDVI_mean'], ['B', 'G', 'R','NIR', 'clusters', 'bathymetry','NDVI'])

# source code: https://gis.stackexchange.com/questions/333413/is-google-earth-engine-snic-segmentation-algorithm-inconsistent. Accessed 10.02.23
snic.reproject(**{
  'crs': 'EPSG:4326',
  'scale': 10
})

# Source code: https://developers.google.com/earth-engine/apidocs/ee-algorithms-image-segmentation-snic. 10.02.23
# Lock map zoom to maintain the desired scale of the segmentation computation.

#Display the clusters.
#Map.addLayer(snic.randomVisualizer(), None, 'Clusters')
# Display the RGB cluster means.
visParams = {
  'bands': ['R', 'G', 'B'],
  'min': 0,
  'max': 255
}
#Map.addLayer(snic, visParams, 'RGB cluster means')

# Compute per-cluster stdDev.
clusters = snic.select('clusters')    # image.addBands(clusters) or S2_deglint.addBands(clusters)
stdDev = S2_deglint.addBands(clusters).reduceConnectedComponents(ee.Reducer.stdDev(), 'clusters', 256)
#Map.addLayer(stdDev, {min:0, max:0.1}, 'StdDev', False)

#Area
area = ee.Image.pixelArea().addBands(clusters).reduceConnectedComponents(ee.Reducer.sum(), 'clusters', 256)
#Map.addLayer(area, {min:50000, max: 500000}, 'Cluster Area', False)

minMax = clusters.reduceNeighborhood(ee.Reducer.minMax(), ee.Kernel.square(1))
perimeterPixels = minMax.select(0).neq(minMax.select(1)).rename('perimeter')
#Map.addLayer(perimeterPixels, {min: 0, max: 1}, 'perimeterPixels')

# Perimeter
perimeter = perimeterPixels.addBands(clusters).reduceConnectedComponents(ee.Reducer.sum(), 'clusters', 256);
#Map.addLayer(perimeter, {min: 100, max: 400}, 'Perimeter size', False);

# Width and Height
sizes = ee.Image.pixelLonLat().addBands(clusters).reduceConnectedComponents(ee.Reducer.minMax(), 'clusters', 256)
width = sizes.select('longitude_max').subtract(sizes.select('longitude_min')).rename('width')
height = sizes.select('latitude_max').subtract(sizes.select('latitude_min')).rename('height')
#Map.addLayer(width, {min:0, max:0.02}, 'Cluster width', False)
#Map.addLayer(height, {min:0, max:0.02}, 'Cluster height', False)

bands = ['R', 'G', 'B', 'NIR','clusters','bathymetry','NDVI']

# objectPropertiesImage is now an image containing 11 feautres.  
objectPropertiesImage = ee.Image.cat([
  snic.select(bands),
  stdDev,
  area,
  perimeter,
  width,
  height
]).float()

### sample data (11 features)

In [ ]:
'''
# Use functions below if data to get sample data fro training, validaiton and test. 
# To save time, these sets have already been created and stored locally.
# The files used her ecan be found with the rest on git-hub for reproducable results.

# increase sample size of seaweed
sample_sw = objectPropertiesImage.sample(**{
  'region': seaweed_shp,
  'numPixels':26000,  # As it does not collect clsoe to 5000 samples when set to this, we increase this value. Should be adjusted if error is thrown.
  'geometries': True,
  'seed' :3,
  'scale': 10,   # Set to 10 as our resolution is 10-meters
})

sample_sg = objectPropertiesImage.sample(**{
  'region': seagrass_shp,
  'numPixels':5000,   # 5000 pixels is the maximum number of pixels that can be processed in GEE
  'geometries': True,
  'seed' :3,
  'scale': 10,
})

sample_water = objectPropertiesImage.sample(**{
  'region': water_shp.geometry(),
  'numPixels':5000,
  'geometries': True,
  'seed' :3,
  'scale': 10,
})

#Code to display sample-points if needed


Map.addLayer(sample_sg, vis_params_sg, 'seagrass samples')
Map.addLayer(sample_sw, vis_params_sw, 'seaweed samples')
Map.addLayer(sample_water, vis_params_water, 'water samples')
Map

# Convert samples from ee-objects to geodataframes
gdf_sw = geemap.ee_to_geopandas(sample_sw)
gdf_sg = geemap.ee_to_geopandas(sample_sg)
gdf_water = geemap.ee_to_geopandas(sample_water)

# Get number of samples fetched for each class
length_sg = len(gdf_sg)
length_sw = len(gdf_sw)
length_water = len(gdf_water)
print(f'We have: \n {length_sg} samples from the seagrass polygons \n {length_sw} samples from the seaweed polygons \n {length_water} samples from the water polygons ')

 # Crete Class column 'Cover'
gdf_water['Cover'] =  int(1) # class water
gdf_sw['Cover'] =  int(2)    # class seaweed
gdf_sg['Cover'] =  int(3)    # class seagrass


# Merge the three dataframes and create a single dataset
# Source code: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html. Accessed 08.02.23
frames = [gdf_water, gdf_sw, gdf_sg]

data = pd.concat(frames, ignore_index=True)
print(len(data))
'''

# Pixel-based (7 feautres) classification

- 1) Import geometries to fetch samples from
- 2) Sample data
- 3) Locally store data for faster processing for next time as well as reproducability


### Import geometries to fetch samples from

In [ ]:
# Add ground truth data geometry of seagrass and seaweed
# Must upload to google colabs for each new runtime...
shp_path_SW = '/content/try1.shp'
seaweed_shp = geemap.shp_to_ee(shp_path_SW)

shp_path_SG = '/content/MaineDEP_-_Eelgrass_2018_(Casco_Bay_Only).shp'
seagrass_shp = geemap.shp_to_ee(shp_path_SG)

# Drew rectangles to represent water, than dowloaded them locally
# geemap.ee_to_shp(water, filename='/content/water.shp')
shp_path_water = '/content/water.shp'
water_shp = geemap.shp_to_ee(shp_path_water)


### Sample data (7 features)

In [ ]:
# Use functions below if data to get sample data fro training, validaiton and test. 
# To save time, these sets have already been created and stored locally.
# The files used her ecan be found with the rest on git-hub for reproducable results.

'''
# Samples of seaweed
sample_sw = S2_deglint.sample(**{
  'region': seaweed_shp,
  'numPixels':25000, # As it does not collect clsoe to 5000 samples when set to this, we increase this value. Should be adjusted if error is thrown.
  'geometries': True,
  'seed' :3,
  'scale': 10,      # Set to 10 as our resolution is 10-meters    
})

# Samples of seagrass
sample_sg = S2_deglint.sample(**{
  'region': seagrass_shp,
  'numPixels':5000,  # 5000 pixels is the maximum number of pixels that can be processed in GEE
  'geometries': True,
  'seed' :3,
  'scale': 10,
})

# Sample of water
sample_water = S2_deglint.sample(**{
  'region': water_shp,
  'numPixels':5000,
  'geometries': True,
  'seed' :3,
  'scale': 10,
})


# Code to display sample-points if needed
Map.addLayer(sample_sg, vis_params_sg, 'seagrass samples')
Map.addLayer(sample_sw, vis_params_sw, 'seaweed samples')
Map.addLayer(sample_water, vis_params_water, 'water samples')

Map

# Convert samples from ee-objects to geodataframes
gdf_sw = geemap.ee_to_geopandas(sample_sw)
gdf_sg = geemap.ee_to_geopandas(sample_sg)
gdf_water = geemap.ee_to_geopandas(sample_water)


# Get number of samples fetched for each class
length_sg = len(gdf_sg)
length_sw = len(gdf_sw)
length_water = len(gdf_water)
print(f'We have: \n {length_sg} samples from the seagrass polygons \n {length_sw} samples from the seaweed polygons \n {length_water} samples from the water polygons ')

 # Crete Class column 'Cover'
gdf_water['Cover'] =  int(1) # class water
gdf_sw['Cover'] =  int(2)    # class seaweed
gdf_sg['Cover'] =  int(3)    # class seagrass


# Merge the three dataframes and create a single dataset
# Source code: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html. Accessed 08.02.23
frames = [gdf_water, gdf_sw, gdf_sg]

data = pd.concat(frames, ignore_index=True)
print(len(data))
'''

### Locally store data for faster processing for next time as well as reproducability

In [ ]:
# How to locally store sample data:

# All data: 
# 1) data = data.drop('geometry', axis=1)
# 2) data.to_csv(r'/content/data_pixel_based_UML_23.csv', index=False)'

# Only training data:
# 1) data = data.sample(frac=1)
# 2) train = data.drop(['geometry','Cover'], axis=1)
# 3) train.to_csv(r'/content/train_pixel_based_UML_23.csv', index=False)

# Only class data:
# 1) y_true = data['Cover']
# 2) y_true.to_csv(r'/content/ytrue_pixel_based_UML_23.csv', index=False)

# Access locally stored data: 
# 1) Upload files to session storage
# 2) Choose training and classs data seperatly:
df = pd.read_csv('/content/train_pixel_based_UML_23.csv')
y_true_df = pd.read_csv('/content/ytrue_pixel_based_UML_23.csv')
data7 = pd.read_csv('/content/data_pixel_based_UML_23.csv')

df.columns
#data.isnull().sum().sum() # Check null-values, in which case must be removed.

# LOCALLY STORING PROCESS FOR OBIA
'''
# How to locally store sample data:

# All data: 
# 1) data.head()               # Check how the data looks 
# 2) data.isnull().sum().sum() # Check for null-values 
# 3) data.columns              # Found mix-up mistake of columns
# 4) data = data.drop(['geometry','B', 'G', 'NDVI_1', 'NIR',
#       'R', 'bathymetry_1'], axis=1) # fixed mistake 
# 5) data.head()                # Verify that our fix worked
# 6) data = data.sample(frac=1) # Randomize the data
# 7) data.to_csv(r'/content/data_OBIA_MT23.csv', index=False)

# Only training data:
# 1) train = data.drop(['Cover'], axis=1)
# 2) train.to_csv(r'/content/train_OBIA_MT23.csv', index=False)

# Only class data:
# 3) y_true = data['Cover']
# 4) y_true.to_csv(r'/content/ytrue_OBIA_MT23.csv', index=False)

# Access locally stored data: 
# 1) Upload files to session storage
# 2) Choose training and classs data seperatly:
df = pd.read_csv('/content/train_OBIA_MT23.csv')
y_true_df = pd.read_csv('/content/ytrue_OBIA_MT23.csv')
data11 = pd.read_csv('/content/data_OBIA_MT23.csv')

df.head()
'''

In [12]:
#Load datasets 

# Data with 11 feautres 
#data11 = pd.read_csv('/content/data_OBIA_MT23.csv')

# Data with 7 feautres 
data7 = pd.read_csv('/content/data_pixel_based_UML_23.csv')

# Data with 3 feautres 
data3 = data7.loc[:,['B2','B3','B4','Cover']]

In [32]:
len(data3.columns)

4

In [ ]:

# Standardize the data 
X_std = StandardScaler().fit_transform(X)
# Training dataset set to 80 %
X_train, X_rem, y_train, y_rem = train_test_split(X_std, y, train_size=0.8, random_state=42)
# Validation and test dataset set to 10% each
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=42)

#Supervised - Random Forest (RF)

In [46]:
def supervised_RF(data):
  '''
  Takes any dataset with X feautres and a column of y-values in.
  Function assumes the last column to contain the class-labels of the data
  '''
  y = data.iloc[:,-1]  # Get class lables
  label = y.name       # Get the name of class label
  X = data.drop(label, axis = 1) # Get only feaures 
  y=y.astype('int')  # y was firstly of type 'object', needs to be an integer for sklearn to recognize it.


  # Standardize the data 
  X_std = StandardScaler().fit_transform(X)
  # Training dataset set to 80 %
  X_train, X_rem, y_train, y_rem = train_test_split(X_std, y, train_size=0.8, random_state=42)
  # Validation and test dataset set to 10% each
  X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=42)

  rfc=RandomForestClassifier(random_state=42)

  param_grid = {'n_estimators':[int(x) for x in np.linspace(start=10, stop = 30, num = 5)],
              'max_features': ['sqrt', 'log2', None],
              'max_depth':[2,4,10],
              'min_samples_split': [2,5],
              'min_samples_leaf':[1,2],
              'bootstrap':[True, False]}

  CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
  CV_rfc.fit(X_train, y_train)
  dict_bp = CV_rfc.best_params_
  rf = ensemble.RandomForestClassifier(n_estimators=dict_bp['n_estimators'],
                                     max_depth=dict_bp['max_depth'],
                                     min_samples_split=dict_bp['min_samples_split'],
                                     min_samples_leaf=dict_bp['min_samples_leaf'],
                                     max_features=dict_bp['max_features'],
                                     bootstrap=dict_bp['bootstrap'],
                                     n_jobs=-1,
                                     random_state=3).fit(X_train, y_train)

  preds = rf.predict(X_valid)
  return accuracy_score(preds, y_valid), len(X.columns)


In [47]:
start_time = time.time()
results = supervised_RF(data3)  
print("Oerall accuracy obtained by RF with {} features is: {:.2f}%".format(results[1], (results[0]*100)))
print('Supervised classificaiotn with {} features completed in: {:.2f} minutes'.format(results[1],((time.time() - start_time)/60)))

Oerall accuracy obtained by RF with 3 features is: 75.49%
Supervised classificaiotn with 3 features completed in: 6.41 minutes


In [91]:
# Load datasets 

# Data with 11 feautres 
data11 = pd.read_csv('/content/data_OBIA_MT23.csv')

# Data with 7 feautres 
data7 = pd.read_csv('/content/data_pixel_based_UML_23.csv')

# Data with 3 feautres 
data3 = data7.loc[:,['B2','B3','B4','Cover']]



In [ ]:
# Loop over data with different numbers of feautres
for j in [data11, data7, data3]:
  start_time = time.time()
  results = supervised_RF(j)
  print("Oerall accuracy obtained by RF with {} features is: {:.2f}%".format(results[1], (results[0]*100)))
  print('Supervised classificaiotn with {} features completed in: {:.2f} minutes'.format(results[1],((time.time() - start_time)/60)))
  print('\n')

In [83]:
def unsupervised(data, pca= True, algorithm = " "):
  y = data.iloc[:,-1]  # Get class lables
  label = y.name       # Get the name of class label
  X = data.drop(label, axis = 1) # Get only feaures 
  y=y.astype('int')  # y was firstly of type 'object', needs to be an integer for sklearn to recognize it.
  X_std = StandardScaler().fit_transform(X)

  if pca == True:
      pca = PCA(n_components = 2)
      X_pca = pca.fit(X_std)
      X_trans_pca = pca.transform(X_std)

      if algorithm == 'kMeans':
        km = KMeans(n_clusters=3, max_iter=100, random_state=40)
        km.fit(X_trans_pca)
        y_pred = km.labels_
      else:
        db = DBSCAN(eps=0.2, min_samples = 5, metric='euclidean')
        db_fit = db.fit_predict(X_trans_pca)
        y_pred = db.labels_
      
  else:
    if algorithm == 'kMeans':
        km = KMeans(n_clusters=3, max_iter=100, random_state=40)
        km.fit(X_std)
        y_pred = km.labels_ # Store predicted class for each point
    else:
        db = DBSCAN(eps=0.2, min_samples = 5, metric='euclidean')
        db_fit = db.fit_predict(X_std)
        y_pred = db.labels_

  return accuracy_score(y, y_pred), len(X.columns)

In [103]:

for j in [data11, data7, data3]:
  start_time = time.time()
  results = unsupervised(j, False, 'kMeans')
  #print(f'Results {j} \n' )
  print("Oerall accuracy obtained by Kmeans with {} features is: {:.2f}%".format(results[1], (results[0]*100)))
  print('Unupervised classificaiotn with {} features completed in: {:.2f} minutes'.format(results[1],((time.time() - start_time)/60)))
  print('\n')


Oerall accuracy obtained by Kmeans with 11 features is: 4.83%
Unupervised classificaiotn with 11 features completed in: 0.02 minutes


Oerall accuracy obtained by Kmeans with 6 features is: 8.87%
Unupervised classificaiotn with 6 features completed in: 0.01 minutes


Oerall accuracy obtained by Kmeans with 3 features is: 0.36%
Unupervised classificaiotn with 3 features completed in: 0.00 minutes




In [104]:
for j in [data11, data7, data3]:
  start_time = time.time()
  results = unsupervised(j, True, 'kMeans')
  #print(f'Results {j} \n' )
  print("Oerall accuracy obtained by Kmeans using 2 PCA components with {} features is: {:.2f}%".format(results[1], (results[0]*100)))
  print('Unupervised classificaiotn with {} features completed in: {:.2f} minutes'.format(results[1],((time.time() - start_time)/60)))
  print('\n')

Oerall accuracy obtained by Kmeans using 2 PCA components with 11 features is: 21.55%
Unupervised classificaiotn with 11 features completed in: 0.01 minutes


Oerall accuracy obtained by Kmeans using 2 PCA components with 6 features is: 30.16%
Unupervised classificaiotn with 6 features completed in: 0.01 minutes


Oerall accuracy obtained by Kmeans using 2 PCA components with 3 features is: 0.35%
Unupervised classificaiotn with 3 features completed in: 0.02 minutes




In [105]:
for j in [data11, data7, data3]:
  start_time = time.time()
  results = unsupervised(j, True, 'DBSCAN')
  #print(f'Results {j} \n' )
  print("Oerall accuracy obtained by DBSCAN using 2 PCA components with {} features is: {:.2f}%".format(results[1], (results[0]*100)))
  print('Unupervised classificaiotn with {} features completed in: {:.2f} minutes'.format(results[1],((time.time() - start_time)/60)))
  print('\n')

Oerall accuracy obtained by DBSCAN using 2 PCA components with 11 features is: 0.24%
Unupervised classificaiotn with 11 features completed in: 0.00 minutes


Oerall accuracy obtained by DBSCAN using 2 PCA components with 6 features is: 0.00%
Unupervised classificaiotn with 6 features completed in: 0.01 minutes


Oerall accuracy obtained by DBSCAN using 2 PCA components with 3 features is: 0.00%
Unupervised classificaiotn with 3 features completed in: 0.01 minutes




In [106]:
for j in [data11, data7, data3]:
  start_time = time.time()
  results = unsupervised(j, False, 'DBSCAN')
  #print(f'Results {j} \n' )
  print("Oerall accuracy obtained by DBSCAN with {} features is: {:.2f}%".format(results[1], (results[0]*100)))
  print('Unupervised classificaiotn with {} features completed in: {:.2f} minutes'.format(results[1],((time.time() - start_time)/60)))
  print('\n')

Oerall accuracy obtained by DBSCAN with 11 features is: 0.34%
Unupervised classificaiotn with 11 features completed in: 0.02 minutes


Oerall accuracy obtained by DBSCAN with 6 features is: 0.62%
Unupervised classificaiotn with 6 features completed in: 0.01 minutes


Oerall accuracy obtained by DBSCAN with 3 features is: 0.01%
Unupervised classificaiotn with 3 features completed in: 0.01 minutes




In [107]:
def semiSupervised(data):
  y = data.iloc[:,-1]  # Get class lables
  label = y.name       # Get the name of class label
  X = data.drop(label, axis = 1) # Get only feaures 
  y=y.astype('int')  # y was firstly of type 'object', needs to be an integer for sklearn to recognize it.
  X_std = StandardScaler().fit_transform(X)

  # Firstly, the dataset is split into a unlabeled and a remaining datasets
  # X_unlabled is 80% of the total dataset
  X_unlabled, X_rem, y_unlabled, y_rem = train_test_split(X_std, y, train_size=0.8, random_state=42)

  # Then, the remaining data is split into a labeled and a test dataset. 
  # we have to define valid_size=0.5 (that is 50% of remaining data)
  # The labled and test datasets each conaints 10% of the samples from the total data 

  test_size = 0.5
  X_labeled, X_test, y_labeled, y_test = train_test_split(X_rem,y_rem, test_size=0.5, random_state=42, shuffle=True)

  # Create a training and validation data from the labled data
  X_train, X_valid, y_train, y_valid = train_test_split(X_labeled,y_labeled, test_size=0.5, random_state=42, shuffle=True)


  # Initialize classifier
  rfc=RandomForestClassifier(random_state=42)
  param_grid = {'n_estimators':[int(x) for x in np.linspace(start=10, stop = 30, num = 5)],
              'max_features': ['sqrt', 'log2', None],
              'max_depth':[2,4,10],
              'min_samples_split': [2,5],
              'min_samples_leaf':[1,2],
              'bootstrap':[True, False]}

  CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
  CV_rfc.fit(X_train, y_train)
  dict_bp = CV_rfc.best_params_
  rf = ensemble.RandomForestClassifier(n_estimators=dict_bp['n_estimators'],
                                     max_depth=dict_bp['max_depth'],
                                     min_samples_split=dict_bp['min_samples_split'],
                                     min_samples_leaf=dict_bp['min_samples_leaf'],
                                     max_features=dict_bp['max_features'],
                                     bootstrap=dict_bp['bootstrap'],
                                     n_jobs=-1,
                                     random_state=3).fit(X_train, y_train)

  preds = rf.predict(X_unlabled)

  # Merge dataset containing the labels
  y_pseudo = np.append(preds, y_train)

  # Merge dataset containing samples 
  pseudo_data = np.append(X_unlabled, X_train, axis=0)

  rf = ensemble.RandomForestClassifier(n_estimators=dict_bp['n_estimators'],
                                     max_depth=dict_bp['max_depth'],
                                     min_samples_split=dict_bp['min_samples_split'],
                                     min_samples_leaf=dict_bp['min_samples_leaf'],
                                     max_features=dict_bp['max_features'],
                                     bootstrap=dict_bp['bootstrap'],
                                     n_jobs=-1,
                                     random_state=3).fit(pseudo_data, y_pseudo)

  preds_pseudo = rf.predict(X_test)
  return accuracy_score(preds_pseudo, y_test), len(X.columns)

for j in [data11, data7, data3]:
  start_time = time.time()
  results = semiSupervised(j)
  #print(f'Results {j} \n' )
  print("Oerall accuracy obtained semisupervised model using RF with {} features is: {:.2f}%".format(results[1], (results[0]*100)))
  print('Unupervised classificaiotn with {} features completed in: {:.2f} minutes'.format(results[1],((time.time() - start_time)/60)))
  print('\n')


Oerall accuracy obtained by DBSCAN with 11 features is: 93.15%
Unupervised classificaiotn with 11 features completed in: 1.19 minutes


Oerall accuracy obtained by DBSCAN with 6 features is: 89.86%
Unupervised classificaiotn with 6 features completed in: 1.02 minutes


Oerall accuracy obtained by DBSCAN with 3 features is: 70.72%
Unupervised classificaiotn with 3 features completed in: 0.92 minutes


